# Recommender Systems 2023/24

### Practice - Hyperparameter optimization with Optuna

### Hyperparameter optimization is essential to achieve the best recommendation quality!!

## How does it work
* Split the data in training, validation and test. Ensure that the distribution of the three sets is similar (e.g., the validation data you use should be split with the same approach used for the test data; if you know the test data contains cold items or users, ensure the validation data does too...)
* Choose a recommender model, identify its hyperparameters and choose a value range and distribution for each.
* Select a hyperparameter configuration and fit your model using the training data. Evaluate it using the validation data.
* Repeat the previous step by exploring many possible hyperparameter configurations. Several exploration strategies are possible.
* Select the hyperparameter configuration with the best recommendation quality on the validation data. Use that configuration to fit the model on the union of training and validation data. Evaluate the final model on the test data and report that result (or submit to the course challenge).

Using directly or indirectly any information on the composition of the test data at any stage of the training or optimization process will result in *information leakage* and cause you to overestimate the quality of your model.



## Hyperparameter optimization strategies

### Grid-search
For each hyperparameter define a list of possible values, then explore *all* possible combinations. For example, in a KNN using tversky similarity you could choose these values:
* number of neighbors: [10, 50, 100, 150, 200, 250]
* shrink term: [0, 10, 20, 50, 100]
* alpha: [0.1, 0.3, 0.5, 0.7, 0.9, 1.1, 1.3, 1.5]
* beta: [0.1, 0.3, 0.5, 0.7, 0.9, 1.1, 1.3, 1.5]

You end up with *1920* possible hyperparameter configurations. Furthermore, you cannot easily fine-tune. Say that you found the optimal number of neighbors is 200, but in that area of the search space you were using a step of 50. What if there is an even better result at 215? You have to define a *new* hyperparameter range around the optimal one you found and use a smaller step, e.g., [180, 190, 200, 210, 220] and so on. 

Overall, grid-search is very sensitive to the range and distribution you choose, very rigid, and often unpractically slow due to how the number of cases grows combinatorially. It has been known for more than 20 years that it is generally a bad idea.

### Random-search
In a random search you define the range and distribution of possible values, and then you pick at random from it each hyperparameter configuration you explore.
For example, in a KNN using tversky similarity you could choose these values:
* number of neighbors: uniform random from 10 to 500
* shrink term: uniform random from 0 to 500
* alpha: uniform random from 0.1 to 1.5
* beta: uniform random from 0.1 to 1.5

It is very parallelizable and effective (definitely better than Grid-search).


### Bayesian-search
More advanced strategy that uses a gaussian process to try to model the interdependencies between hyperparameters based on how they affect the result. It contains a part of random search where the search space is explored, then a second part that uses the gaussian process to choose the next hyperparameter configuration. It combines exploration with exploitation. 

Bayesian-search is less parallelizable (the gaussian process is sequential) but has a good exploration-exploitation tradeoff. It is the strategy used in our research activity.


### Other strategies
There is no shortage of hyperparameter optimization strategies, for example the Tree-Structured Parzen Estimator with Optuna (in this notebook), as well as techniques that use previous optimization runs to attempt transfer learning between models and datasets. All those are beyond the scope of the course.



### In the course repository you will find a BayesianSearch object in the HyperparameterTuning folder. That is a simple wrapper of another library and its purpose is to provide a very simple way to tune some of the most common hyperparameters. 




## Hyperparameter sensitivity

One of the most important problems you encounter when doing hyperparameter tuning is how to select the range and distribution of the hyperparameters. There is no universal rule and some of those decisions are based on experience. Generally the suggestions we can give you are:
* Keep the search space on the larger side, better have more room to manouver than less.
* Hyperparameters like the number of neighbors, the shrink term, the number of latent dimensions of matrix factorization models etc. work well with a uniform random distribution
* Hyperparameters like the learning rate and l1 l2 regularizations work better with a log-uniform distribution as the training tends to be affected by the orders of magnitude rather than the absolute values. You need to be able to explore maybe from 10^-9 to 10^-3, but is less important to choose the specific value.

### In the course repository you will find a script called run_hyperparameter_search that contains a list of commonly used hyperparameters with the corresponding range and distribution.

### Hint: If you see that the optimization yelds a hyperparameter that has a value at either end of the range (min or max) this may indicate that in that scenarion you need to expand the search space.



## Early-stopping

A special hyperparameter is the number of epochs you should train a machine learning model for.
It is possible to put this number as a hyperparameter, but that is not a super-effective strategy. A common strategy is *early-stopping*. It works as follows:
* Select a maximum number of epochs, say 500
* Train the model for a certain number of epochs, say 5
* Evaluate the recommendation quality of the model on the validation data. Create a clone of the model.
* Continue training and evaluate the model periodically. Every time you find a better recommendation quality update the model clone, which will then represent the "best" model you have.
* If the recommendation quality does not improve for a certain number of consecutive validation steps, say 5, stop the training. If you reach the maximum number of epochs, stop the training.
* Use the "best" model clone to generate the recommendations on the validation data.
* When you train the model on the union of training and validation data, use the optimal number of epochs selected by the early-stopping at the previous step.

Usually early-stopping allows to save a lot of computational time and fine-tune the optimal number of epochs, unless the validation step takes a very large amount of time. In those cases, it can be better to either use early-stopping on the algorithm loss function (which is however a different problem then recommendation and does not guarantee you that the best absolute loss will correspond to the model with the best recommendation quality) or just select the number of epochs as any other hyperparameter.


In [1]:
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Data_manager.Movielens.Movielens1MReader import Movielens1MReader

data_reader = Movielens1MReader()
data_loaded = data_reader.load_data()

URM_all = data_loaded.get_URM_all()
ICM_all = data_loaded.get_ICM_from_name("ICM_genres")

Movielens1M: Verifying data consistency...
Movielens1M: Verifying data consistency... Passed!
DataReader: current dataset is: Movielens1M
	Number of items: 3883
	Number of users: 6040
	Number of interactions in URM_all: 1000209
	Value range in URM_all: 1.00-5.00
	Interaction density: 4.26E-02
	Interactions per user:
		 Min: 2.00E+01
		 Avg: 1.66E+02
		 Max: 2.31E+03
	Interactions per item:
		 Min: 0.00E+00
		 Avg: 2.58E+02
		 Max: 3.43E+03
	Gini Index: 0.53

	ICM name: ICM_genres, Value range: 1.00 / 1.00, Num features: 18, feature occurrences: 6408, density 9.17E-02
	ICM name: ICM_year, Value range: 1.92E+03 / 2.00E+03, Num features: 1, feature occurrences: 3883, density 1.00E+00


	UCM name: UCM_all, Value range: 1.00 / 1.00, Num features: 3469, feature occurrences: 24160, density 1.15E-03


### How do we perform hyperparameter optimization?
* Split the data in three *disjoint* sets: training, validation and testing data
* Define a set of hyperparameters with the range and distribution
* Explore hyperparameter space and select those with the best recommendation quality on the *validation* data (including the number of epochs for ML algorithms)
* Given the best hyperparameters, fit the model again using the union of training and validation data.
* Evaluate this last model on the testing data.

### Step 1: Split the data and create the evaluator objects

In [2]:
from Evaluation.Evaluator import EvaluatorHoldout

URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.8)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 18 ( 0.3%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 4 ( 0.1%) Users that have less than 1 test interactions


### Step 2: Define the objective function that will be run by the optimizer

The obective function should:
* Use the optuna trial instance to acquire the hyperparameter values required
* Fit the model
* Evaluate on the validation and return the result of a desired metric

In [3]:
import optuna
import pandas as pd
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender

def objective_function(optuna_trial):
    
    recommender_instance = ItemKNNCFRecommender(URM_train)
    recommender_instance.fit(topK = optuna_trial.suggest_int("topK", 5, 1000),
                             shrink = optuna_trial.suggest_int("shrink", 0, 1000),
                             similarity = "cosine",
                             normalize = optuna_trial.suggest_categorical("normalize", [True, False])
                            )
    
    result_df, _ = evaluator_validation.evaluateRecommender(recommender_instance)
    
    return result_df.loc[10]["MAP"]

### Step 3: Create an object/function that will be called after every call of the objective function to log the results in a dataframe for easier readability

In [4]:
class SaveResults(object):
    
    def __init__(self):
        self.results_df = pd.DataFrame()
    
    def __call__(self, optuna_study, optuna_trial):
        hyperparam_dict = optuna_trial.params.copy()
        hyperparam_dict["result"] = optuna_trial.values[0]
        
        self.results_df = self.results_df.append(hyperparam_dict, ignore_index=True)

### Step 4: Create an optuna Study and run

In [5]:
optuna_study = optuna.create_study(direction="maximize")
        
save_results = SaveResults()
        
optuna_study.optimize(objective_function,
                      callbacks=[save_results],
                      n_trials = 50)

[I 2023-11-23 12:35:34,503] A new study created in memory with name: no-name-b3885cb4-b6a0-49a6-8a67-fb9cb8aba7b7


ItemKNNCFRecommender: URM Detected 226 ( 5.8%) items with no interactions.
Similarity column 3883 (100.0%), 3362.37 column/sec. Elapsed time 1.15 sec
EvaluatorHoldout: Processed 6022 (100.0%) in 5.85 sec. Users per second: 1030


[I 2023-11-23 12:35:41,693] Trial 0 finished with value: 0.10099880464750927 and parameters: {'topK': 718, 'shrink': 179, 'normalize': True}. Best is trial 0 with value: 0.10099880464750927.


ItemKNNCFRecommender: URM Detected 226 ( 5.8%) items with no interactions.
Similarity column 3883 (100.0%), 3716.29 column/sec. Elapsed time 1.04 sec
EvaluatorHoldout: Processed 6022 (100.0%) in 5.81 sec. Users per second: 1036


[I 2023-11-23 12:35:48,669] Trial 1 finished with value: 0.07348513782796438 and parameters: {'topK': 373, 'shrink': 77, 'normalize': False}. Best is trial 0 with value: 0.10099880464750927.


...

[I 2023-11-23 12:41:32,903] Trial 47 finished with value: 0.10472606395597076 and parameters: {'topK': 240, 'shrink': 783, 'normalize': True}. Best is trial 34 with value: 0.10477512348900538.


ItemKNNCFRecommender: URM Detected 226 ( 5.8%) items with no interactions.
Similarity column 3883 (100.0%), 3686.96 column/sec. Elapsed time 1.05 sec
EvaluatorHoldout: Processed 6022 (100.0%) in 6.39 sec. Users per second: 943


[I 2023-11-23 12:41:40,454] Trial 48 finished with value: 0.10392322726721537 and parameters: {'topK': 214, 'shrink': 769, 'normalize': True}. Best is trial 34 with value: 0.10477512348900538.


ItemKNNCFRecommender: URM Detected 226 ( 5.8%) items with no interactions.
Similarity column 3883 (100.0%), 2657.06 column/sec. Elapsed time 1.46 sec
EvaluatorHoldout: Processed 6022 (100.0%) in 6.06 sec. Users per second: 994


[I 2023-11-23 12:41:48,127] Trial 49 finished with value: 0.10461281518031691 and parameters: {'topK': 253, 'shrink': 669, 'normalize': True}. Best is trial 34 with value: 0.10477512348900538.


### Step 3: Given the best hyperparameters, train on the union of URM_train and validation, to evaluate on test

In [6]:
pruned_trials = [t for t in optuna_study.trials if t.state == optuna.trial.TrialState.PRUNED]
complete_trials = [t for t in optuna_study.trials if t.state == optuna.trial.TrialState.COMPLETE]

print("Study statistics: ")
print("  Number of finished trials: ", len(optuna_study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
print("  Value Validation: ", optuna_study.best_trial.value)


Study statistics: 
  Number of finished trials:  50
  Number of pruned trials:  0
  Number of complete trials:  50
Best trial:
  Value Validation:  0.10477512348900538


In [7]:
optuna_study.best_trial

FrozenTrial(number=34, state=TrialState.COMPLETE, values=[0.10477512348900538], datetime_start=datetime.datetime(2023, 11, 23, 12, 39, 46, 283471), datetime_complete=datetime.datetime(2023, 11, 23, 12, 39, 53, 389138), params={'topK': 242, 'shrink': 991, 'normalize': True}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'topK': IntDistribution(high=1000, log=False, low=5, step=1), 'shrink': IntDistribution(high=1000, log=False, low=0, step=1), 'normalize': CategoricalDistribution(choices=(True, False))}, trial_id=34, value=None)

In [8]:
optuna_study.best_trial.params

{'topK': 242, 'shrink': 991, 'normalize': True}

In [9]:
save_results.results_df

,normalize,result,shrink,topK
0,1.0,0.100999,179.0,718.0
1,0.0,0.073485,77.0,373.0
2,0.0,0.072073,441.0,513.0
3,1.0,0.097096,654.0,33.0
4,0.0,0.076444,571.0,198.0
5,0.0,0.073276,741.0,396.0
6,0.0,0.072164,263.0,525.0
7,0.0,0.075008,596.0,273.0
8,1.0,0.098580,379.0,927.0
9,0.0,0.074458,374.0,316.0


In [10]:
recommender_instance = ItemKNNCFRecommender(URM_train + URM_validation)
recommender_instance.fit(**optuna_study.best_trial.params)

result_df, _ = evaluator_test.evaluateRecommender(recommender_instance)
result_df

ItemKNNCFRecommender: URM Detected 195 ( 5.0%) items with no interactions.
Similarity column 3883 (100.0%), 2077.28 column/sec. Elapsed time 1.87 sec
EvaluatorHoldout: Processed 6036 (100.0%) in 6.91 sec. Users per second: 874


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.329225,0.352461,0.145158,0.238979,0.250529,0.611125,0.302173,0.201481,0.871107,1.145516,...,0.999338,0.87053,0.999338,0.042906,7.739146,0.992967,0.122672,0.71569,1.857702,0.064754


### Optuna supports various tools such as a database to store the results which can be used for nice visualizations, try them!


Optuna also supports conditional hyperparameters, which may be required only based on another hyperparameter value. In the following function several similarity heuristics are optimized.

In [11]:
def objective_function_KNN_similarities(optuna_trial):
    
    recommender_instance = ItemKNNCFRecommender(URM_train)
    similarity = optuna_trial.suggest_categorical("similarity", ['cosine', 'dice', 'jaccard', 'asymmetric', 'tversky', 'euclidean'])
    
    full_hyperp = {"similarity": similarity,
                   "topK": optuna_trial.suggest_int("topK", 5, 1000),
                   "shrink": optuna_trial.suggest_int("shrink", 0, 1000),
                  }
    
    if similarity == "asymmetric":
        full_hyperp["asymmetric_alpha"] = optuna_trial.suggest_float("asymmetric_alpha", 0, 2, log=False)
        full_hyperp["normalize"] = True     

    elif similarity == "tversky":
        full_hyperp["tversky_alpha"] = optuna_trial.suggest_float("tversky_alpha", 0, 2, log=False)
        full_hyperp["tversky_beta"] = optuna_trial.suggest_float("tversky_beta", 0, 2, log=False)
        full_hyperp["normalize"] = True 

    elif similarity == "euclidean":
        full_hyperp["normalize_avg_row"] = optuna_trial.suggest_categorical("normalize_avg_row", [True, False])
        full_hyperp["similarity_from_distance_mode"] = optuna_trial.suggest_categorical("similarity_from_distance_mode", ["lin", "log", "exp"])
        full_hyperp["normalize"] = optuna_trial.suggest_categorical("normalize", [True, False])
        
    
    recommender_instance.fit(**full_hyperp)
    
    result_df, _ = evaluator_validation.evaluateRecommender(recommender_instance)
    
    return result_df.loc[10]["MAP"]

In [12]:
optuna_study = optuna.create_study(direction="maximize")
        
save_results = SaveResults()
        
optuna_study.optimize(objective_function_KNN_similarities,
                      callbacks=[save_results],
                      n_trials = 50)

[I 2023-11-23 12:41:57,245] A new study created in memory with name: no-name-cd123d42-8533-4abc-8c9d-c3afcd6a3c46


ItemKNNCFRecommender: URM Detected 226 ( 5.8%) items with no interactions.
Similarity column 3883 (100.0%), 3203.17 column/sec. Elapsed time 1.21 sec
EvaluatorHoldout: Processed 6022 (100.0%) in 7.08 sec. Users per second: 851


[I 2023-11-23 12:42:05,672] Trial 0 finished with value: 0.014323307923855902 and parameters: {'similarity': 'asymmetric', 'topK': 315, 'shrink': 548, 'asymmetric_alpha': 1.5538276977920127}. Best is trial 0 with value: 0.014323307923855902.


ItemKNNCFRecommender: URM Detected 226 ( 5.8%) items with no interactions.
Similarity column 3883 (100.0%), 2767.90 column/sec. Elapsed time 1.40 sec
EvaluatorHoldout: Processed 6022 (100.0%) in 6.43 sec. Users per second: 937


[I 2023-11-23 12:42:13,752] Trial 1 finished with value: 0.09779433742942545 and parameters: {'similarity': 'tversky', 'topK': 547, 'shrink': 151, 'tversky_alpha': 0.7381058346171128, 'tversky_beta': 0.9018425361247335}. Best is trial 1 with value: 0.09779433742942545.


...

[I 2023-11-23 12:48:15,593] Trial 47 finished with value: 0.10475646834622215 and parameters: {'similarity': 'asymmetric', 'topK': 293, 'shrink': 155, 'asymmetric_alpha': 0.11203022243580418}. Best is trial 43 with value: 0.10616575203091284.


ItemKNNCFRecommender: URM Detected 226 ( 5.8%) items with no interactions.
Similarity column 3883 (100.0%), 4075.28 column/sec. Elapsed time 0.95 sec
EvaluatorHoldout: Processed 6022 (100.0%) in 5.70 sec. Users per second: 1057


[I 2023-11-23 12:48:22,383] Trial 48 finished with value: 0.10323093762025944 and parameters: {'similarity': 'asymmetric', 'topK': 196, 'shrink': 863, 'asymmetric_alpha': 0.28670516289944026}. Best is trial 43 with value: 0.10616575203091284.


ItemKNNCFRecommender: URM Detected 226 ( 5.8%) items with no interactions.
Similarity column 3883 (100.0%), 2451.57 column/sec. Elapsed time 1.58 sec
EvaluatorHoldout: Processed 6022 (100.0%) in 7.20 sec. Users per second: 836


[I 2023-11-23 12:48:31,401] Trial 49 finished with value: 0.09858953018825156 and parameters: {'similarity': 'dice', 'topK': 352, 'shrink': 568}. Best is trial 43 with value: 0.10616575203091284.


In [13]:
save_results.results_df

,asymmetric_alpha,result,shrink,similarity,topK,tversky_alpha,tversky_beta,normalize,normalize_avg_row,similarity_from_distance_mode
0,1.553828,0.014323,548.0,asymmetric,315.0,NaN,NaN,NaN,NaN,NaN
1,NaN,0.097794,151.0,tversky,547.0,0.738106,0.901843,NaN,NaN,NaN
2,NaN,0.097225,773.0,cosine,982.0,NaN,NaN,NaN,NaN,NaN
3,NaN,0.098334,115.0,dice,231.0,NaN,NaN,NaN,NaN,NaN
4,1.807630,0.014564,837.0,asymmetric,895.0,NaN,NaN,NaN,NaN,NaN
5,NaN,0.085886,72.0,euclidean,569.0,NaN,NaN,0.0,1.0,lin
6,NaN,0.093072,758.0,jaccard,638.0,NaN,NaN,NaN,NaN,NaN
7,1.691509,0.011258,286.0,asymmetric,230.0,NaN,NaN,NaN,NaN,NaN
8,NaN,0.101108,526.0,cosine,610.0,NaN,NaN,NaN,NaN,NaN
9,NaN,0.094858,922.0,dice,498.0,NaN,NaN,NaN,NaN,NaN


In [14]:
optuna_study.best_trial.params

{'similarity': 'asymmetric',
 'topK': 192,
 'shrink': 110,
 'asymmetric_alpha': 0.15610763063806515}

In [15]:
recommender_instance = ItemKNNCFRecommender(URM_train + URM_validation)
recommender_instance.fit(**optuna_study.best_trial.params)

result_df, _ = evaluator_test.evaluateRecommender(recommender_instance)
result_df

ItemKNNCFRecommender: URM Detected 195 ( 5.0%) items with no interactions.
Similarity column 3883 (100.0%), 2154.97 column/sec. Elapsed time 1.80 sec
EvaluatorHoldout: Processed 6036 (100.0%) in 5.76 sec. Users per second: 1049


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.334675,0.35819,0.14819,0.243029,0.254891,0.621265,0.316909,0.205422,0.876408,1.163663,...,0.999338,0.875828,0.999338,0.039911,7.661528,0.992851,0.114108,0.708512,1.837124,0.06488


### An example with earlystopping, for SVD++

When training machine learning models that require iterative training for a certain number of epochs one also has to select this value. There are two strategies:
* Consider the number of epochs as another hyperparameter to be determined by our optimizer
* Fix a maximum number of epochs and periodically evaluate the model during the training process. If the recommendation quality does not improve after a certain number of epochs (20, 50 ...) stop the training and select the number of epochs that gave the best result. This is called earlystopping.

In [16]:
from Recommenders.MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_SVDpp_Cython

class SaveResults(object):
    
    def __init__(self):
        self.results_df = pd.DataFrame()
    
    def __call__(self, optuna_study, optuna_trial):
        hyperparam_dict = optuna_trial.params.copy()
        hyperparam_dict["result"] = optuna_trial.values[0]
        
        # Retrieve the optimal number of epochs from the "user attributes" of the trial
        hyperparam_dict["epochs"] = optuna_trial.user_attrs["epochs"]
        
        self.results_df = self.results_df.append(hyperparam_dict, ignore_index=True)
        
        
def objective_function_funksvd(optuna_trial):

    # Earlystopping hyperparameters available in the framework
    full_hyperp = {"validation_every_n": 5,
                   "stop_on_validation": True,
                   "evaluator_object": evaluator_validation,
                   "lower_validations_allowed": 5,   # Higher values will result in a more "patient" earlystopping
                   "validation_metric": "MAP",
                   
                   # MAX number of epochs (usually 500)
                   "epochs": 100,
                  }
                          
        
    recommender_instance = MatrixFactorization_SVDpp_Cython(URM_train)
    recommender_instance.fit(num_factors = optuna_trial.suggest_int("num_factors", 1, 200),
                             sgd_mode = optuna_trial.suggest_categorical("sgd_mode", ["sgd", "adagrad", "adam"]),
                             batch_size = optuna_trial.suggest_categorical("batch_size", [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024]),
                             item_reg = optuna_trial.suggest_float("item_reg", 1e-5, 1e-2, log=True),
                             user_reg =  optuna_trial.suggest_float("user_reg", 1e-5, 1e-2, log=True),
                             learning_rate = optuna_trial.suggest_float("learning_rate", 1e-4, 1e-1, log=True),
                             ** full_hyperp)
    
    # Add the number of epochs selected by earlystopping as a "user attribute" of the optuna trial
    epochs = recommender_instance.get_early_stopping_final_epochs_dict()["epochs"]
    optuna_trial.set_user_attr("epochs", epochs) 
        
    result_df, _ = evaluator_validation.evaluateRecommender(recommender_instance)
    
    return result_df.loc[10]["MAP"]

In [17]:
optuna_study = optuna.create_study(direction="maximize")
        
save_results = SaveResults()
        
optuna_study.optimize(objective_function_funksvd,
                      callbacks=[save_results],
                      n_trials = 10)

[I 2023-11-23 12:48:39,246] A new study created in memory with name: no-name-7c709e2b-b298-40f9-a178-9d5a86011374


MatrixFactorization_SVDpp_Cython_Recommender: URM Detected 226 ( 5.8%) items with no interactions.
SVD++: Processed 640192 (100.0%) in 2.63 sec. MSE loss 1.74E+00. Sample per second: 243002
SVD++: Epoch 1 of 100. Elapsed time 2.30 sec
SVD++: Processed 640192 (100.0%) in 2.84 sec. MSE loss 1.22E+00. Sample per second: 225764
SVD++: Epoch 2 of 100. Elapsed time 4.50 sec
SVD++: Processed 640192 (100.0%) in 3.51 sec. MSE loss 1.21E+00. Sample per second: 182456
SVD++: Epoch 3 of 100. Elapsed time 7.17 sec
SVD++: Processed 640192 (100.0%) in 2.74 sec. MSE loss 1.20E+00. Sample per second: 233230
SVD++: Epoch 4 of 100. Elapsed time 9.42 sec
SVD++: Processed 640192 (100.0%) in 3.52 sec. MSE loss 1.20E+00. Sample per second: 181718
SVD++: Validation begins...
EvaluatorHoldout: Processed 6022 (100.0%) in 4.94 sec. Users per second: 1219
SVD++: CUTOFF: 10 - PRECISION: 0.0527400, PRECISION_RECALL_MIN_DEN: 0.0594916, RECALL: 0.0290703, MAP: 0.0243628, MAP_MIN_DEN: 0.0273094, MRR: 0.1790753, NDCG: 

SVD++: Processed 640192 (100.0%) in 3.26 sec. MSE loss 1.10E+00. Sample per second: 196436
SVD++: Epoch 28 of 100. Elapsed time 1.47 min
SVD++: Processed 640192 (100.0%) in 2.31 sec. MSE loss 1.10E+00. Sample per second: 277227
SVD++: Epoch 29 of 100. Elapsed time 1.50 min
SVD++: Processed 640192 (100.0%) in 2.97 sec. MSE loss 1.10E+00. Sample per second: 215772
SVD++: Validation begins...
EvaluatorHoldout: Processed 6022 (100.0%) in 5.02 sec. Users per second: 1201
SVD++: CUTOFF: 10 - PRECISION: 0.1016772, PRECISION_RECALL_MIN_DEN: 0.1132532, RECALL: 0.0535856, MAP: 0.0443739, MAP_MIN_DEN: 0.0480764, MRR: 0.2280551, NDCG: 0.0982032, F1: 0.0701834, HIT_RATE: 0.5524743, ARHR_ALL_HITS: 0.3143939, NOVELTY: 0.0225779, AVERAGE_POPULARITY: 0.6899390, DIVERSITY_MEAN_INTER_LIST: 0.4620856, DIVERSITY_HERFINDAHL: 0.9462009, COVERAGE_ITEM: 0.0236930, COVERAGE_ITEM_HIT: 0.0172547, ITEMS_IN_GT: 0.8913211, COVERAGE_USER: 0.9970199, COVERAGE_USER_HIT: 0.5508278, USERS_IN_GT: 0.9970199, DIVERSITY_GINI

[I 2023-11-23 12:51:09,141] Trial 0 finished with value: 0.045424725741065916 and parameters: {'num_factors': 44, 'sgd_mode': 'sgd', 'batch_size': 64, 'item_reg': 2.4501996875522273e-05, 'user_reg': 0.00496971633560805, 'learning_rate': 0.001326575440981296}. Best is trial 0 with value: 0.045424725741065916.


MatrixFactorization_SVDpp_Cython_Recommender: URM Detected 226 ( 5.8%) items with no interactions.
SVD++: Processed 640160 (100.0%) in 26.52 sec. MSE loss 8.31E-01. Sample per second: 24136
SVD++: Epoch 1 of 100. Elapsed time 26.24 sec
SVD++: Processed 640160 (100.0%) in 26.40 sec. MSE loss 3.19E-01. Sample per second: 24251
SVD++: Epoch 2 of 100. Elapsed time 52.12 sec
SVD++: Processed 640160 (100.0%) in 27.09 sec. MSE loss 2.22E-01. Sample per second: 23634
SVD++: Epoch 3 of 100. Elapsed time 1.31 min
SVD++: Processed 640160 (100.0%) in 27.25 sec. MSE loss 1.71E-01. Sample per second: 23488
SVD++: Epoch 4 of 100. Elapsed time 1.77 min
SVD++: Processed 640160 (100.0%) in 27.61 sec. MSE loss 1.41E-01. Sample per second: 23185
SVD++: Validation begins...
EvaluatorHoldout: Processed 6022 (100.0%) in 5.55 sec. Users per second: 1086
SVD++: CUTOFF: 10 - PRECISION: 0.0550980, PRECISION_RECALL_MIN_DEN: 0.0605409, RECALL: 0.0263899, MAP: 0.0226386, MAP_MIN_DEN: 0.0244400, MRR: 0.1488346, NDCG

...

MatrixFactorization_SVDpp_Cython_Recommender: URM Detected 226 ( 5.8%) items with no interactions.
SVD++: Processed 640160 (100.0%) in 3.18 sec. MSE loss 4.50E+00. Sample per second: 201467
SVD++: Epoch 1 of 100. Elapsed time 3.11 sec
SVD++: Processed 640160 (100.0%) in 4.17 sec. MSE loss 1.28E+00. Sample per second: 153373
SVD++: Epoch 2 of 100. Elapsed time 7.11 sec
SVD++: Processed 640160 (100.0%) in 3.69 sec. MSE loss 1.23E+00. Sample per second: 173626
SVD++: Epoch 3 of 100. Elapsed time 10.62 sec
SVD++: Processed 640160 (100.0%) in 4.23 sec. MSE loss 1.23E+00. Sample per second: 151206
SVD++: Epoch 4 of 100. Elapsed time 14.17 sec
SVD++: Processed 640160 (100.0%) in 4.20 sec. MSE loss 1.23E+00. Sample per second: 152430
SVD++: Validation begins...
EvaluatorHoldout: Processed 6022 (100.0%) in 4.75 sec. Users per second: 1267
SVD++: CUTOFF: 10 - PRECISION: 0.0098804, PRECISION_RECALL_MIN_DEN: 0.0107281, RECALL: 0.0042535, MAP: 0.0032094, MAP_MIN_DEN: 0.0034635, MRR: 0.0296809, NDCG

SVD++: Processed 640160 (100.0%) in 4.66 sec. MSE loss 1.20E+00. Sample per second: 137423
SVD++: Epoch 28 of 100. Elapsed time 2.11 min
SVD++: Processed 640160 (100.0%) in 4.62 sec. MSE loss 1.20E+00. Sample per second: 138443
SVD++: Epoch 29 of 100. Elapsed time 2.18 min
SVD++: Processed 640160 (100.0%) in 3.74 sec. MSE loss 1.20E+00. Sample per second: 171250
SVD++: Validation begins...
EvaluatorHoldout: Processed 6022 (100.0%) in 5.44 sec. Users per second: 1106
SVD++: CUTOFF: 10 - PRECISION: 0.0336433, PRECISION_RECALL_MIN_DEN: 0.0379345, RECALL: 0.0184269, MAP: 0.0148839, MAP_MIN_DEN: 0.0165913, MRR: 0.1192006, NDCG: 0.0356325, F1: 0.0238117, HIT_RATE: 0.2681833, ARHR_ALL_HITS: 0.1331112, NOVELTY: 0.0289812, AVERAGE_POPULARITY: 0.2662388, DIVERSITY_MEAN_INTER_LIST: 0.9569341, DIVERSITY_HERFINDAHL: 0.9956775, COVERAGE_ITEM: 0.9436003, COVERAGE_ITEM_HIT: 0.0885913, ITEMS_IN_GT: 0.8913211, COVERAGE_USER: 0.9970199, COVERAGE_USER_HIT: 0.2673841, USERS_IN_GT: 0.9970199, DIVERSITY_GINI

SVD++: Validation begins...
EvaluatorHoldout: Processed 6022 (100.0%) in 4.96 sec. Users per second: 1214
SVD++: CUTOFF: 10 - PRECISION: 0.0687811, PRECISION_RECALL_MIN_DEN: 0.0773675, RECALL: 0.0381940, MAP: 0.0319913, MAP_MIN_DEN: 0.0354028, MRR: 0.2075507, NDCG: 0.0734290, F1: 0.0491147, HIT_RATE: 0.4521754, ARHR_ALL_HITS: 0.2566851, NOVELTY: 0.0250551, AVERAGE_POPULARITY: 0.5164967, DIVERSITY_MEAN_INTER_LIST: 0.7823188, DIVERSITY_HERFINDAHL: 0.9782189, COVERAGE_ITEM: 0.7800670, COVERAGE_ITEM_HIT: 0.0612928, ITEMS_IN_GT: 0.8913211, COVERAGE_USER: 0.9970199, COVERAGE_USER_HIT: 0.4508278, USERS_IN_GT: 0.9970199, DIVERSITY_GINI: 0.1208627, SHANNON_ENTROPY: 7.6092835, RATIO_DIVERSITY_HERFINDAHL: 0.9790174, RATIO_DIVERSITY_GINI: 0.3456310, RATIO_SHANNON_ENTROPY: 0.7037140, RATIO_AVERAGE_POPULARITY: 1.8541926, RATIO_NOVELTY: 0.0848950, 

SVD++: New best model found! Updating.
SVD++: Epoch 55 of 100. Elapsed time 4.24 min
SVD++: Processed 640160 (100.0%) in 4.19 sec. MSE loss 1.18E+00. Sam

SVD++: Processed 640160 (100.0%) in 3.78 sec. MSE loss 1.16E+00. Sample per second: 169257
SVD++: Epoch 81 of 100. Elapsed time 6.23 min
SVD++: Processed 640160 (100.0%) in 4.70 sec. MSE loss 1.16E+00. Sample per second: 136221
SVD++: Epoch 82 of 100. Elapsed time 6.29 min
SVD++: Processed 640160 (100.0%) in 4.58 sec. MSE loss 1.16E+00. Sample per second: 139641
SVD++: Epoch 83 of 100. Elapsed time 6.36 min
SVD++: Processed 640160 (100.0%) in 3.63 sec. MSE loss 1.16E+00. Sample per second: 176381
SVD++: Epoch 84 of 100. Elapsed time 6.41 min
SVD++: Processed 640160 (100.0%) in 4.58 sec. MSE loss 1.16E+00. Sample per second: 139896
SVD++: Validation begins...
EvaluatorHoldout: Processed 6022 (100.0%) in 5.06 sec. Users per second: 1190
SVD++: CUTOFF: 10 - PRECISION: 0.0900697, PRECISION_RECALL_MIN_DEN: 0.1005716, RECALL: 0.0483269, MAP: 0.0406923, MAP_MIN_DEN: 0.0445974, MRR: 0.2275887, NDCG: 0.0907179, F1: 0.0629031, HIT_RATE: 0.5214215, ARHR_ALL_HITS: 0.3020112, NOVELTY: 0.0231046, AV

[I 2023-11-23 14:30:21,922] Trial 9 finished with value: 0.04257860727596708 and parameters: {'num_factors': 67, 'sgd_mode': 'sgd', 'batch_size': 32, 'item_reg': 0.002039632392639453, 'user_reg': 0.0009429411014038028, 'learning_rate': 0.00010328797507357167}. Best is trial 3 with value: 0.04762713305182601.


In [18]:
optuna_study.best_trial.params

{'num_factors': 48,
 'sgd_mode': 'adam',
 'batch_size': 128,
 'item_reg': 0.0029531550842413195,
 'user_reg': 0.0005378698094965328,
 'learning_rate': 0.0001832167942980042}

In [19]:
optuna_study.best_trial

FrozenTrial(number=3, state=TrialState.COMPLETE, values=[0.04762713305182601], datetime_start=datetime.datetime(2023, 11, 23, 13, 14, 34, 392523), datetime_complete=datetime.datetime(2023, 11, 23, 13, 19, 6, 122882), params={'num_factors': 48, 'sgd_mode': 'adam', 'batch_size': 128, 'item_reg': 0.0029531550842413195, 'user_reg': 0.0005378698094965328, 'learning_rate': 0.0001832167942980042}, user_attrs={'epochs': 5}, system_attrs={}, intermediate_values={}, distributions={'num_factors': IntDistribution(high=200, log=False, low=1, step=1), 'sgd_mode': CategoricalDistribution(choices=('sgd', 'adagrad', 'adam')), 'batch_size': CategoricalDistribution(choices=(1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024)), 'item_reg': FloatDistribution(high=0.01, log=True, low=1e-05, step=None), 'user_reg': FloatDistribution(high=0.01, log=True, low=1e-05, step=None), 'learning_rate': FloatDistribution(high=0.1, log=True, low=0.0001, step=None)}, trial_id=3, value=None)

In [20]:
save_results.results_df

,batch_size,epochs,item_reg,learning_rate,num_factors,result,sgd_mode,user_reg
0,64.0,20.0,0.000025,0.001327,44.0,0.045425,sgd,0.004970
1,32.0,5.0,0.004023,0.051815,199.0,0.022639,adagrad,0.000813
2,16.0,5.0,0.000351,0.000503,110.0,0.032019,adam,0.000015
3,128.0,5.0,0.002953,0.000183,48.0,0.047627,adam,0.000538
4,32.0,5.0,0.000894,0.020004,35.0,0.029640,sgd,0.000024
5,1024.0,100.0,0.008118,0.000326,151.0,0.006981,adagrad,0.000068
6,8.0,5.0,0.000760,0.000682,67.0,0.028966,adam,0.000015
7,1024.0,100.0,0.000484,0.000496,26.0,0.012343,sgd,0.000490
8,4.0,95.0,0.004535,0.000983,78.0,0.042036,adagrad,0.000031
9,32.0,100.0,0.002040,0.000103,67.0,0.042579,sgd,0.000943
